In [ ]:
# Send message to confluent Kafka topic via python

In [ ]:
# Required connection configs for Kafka producer, consumer, and admin
bootstrap.servers=pkc-619z3.us-east1.gcp.confluent.cloud:9092
security.protocol=SASL_SSL
sasl.mechanisms=PLAIN
sasl.username=7K4SVPMLQXL5AL3A
sasl.password=5XTw3jiNzKiLQLmv0nAmqbQvwgQS/BKD1yF9NchOWAZrj6NrPtMsz5LoZ7DPZ9Vt

# Best practice for higher availability in librdkafka clients prior to 1.7
session.timeout.ms=45000

client.id=ccloud-python-client-b18b45dd-cd5b-4861-83c8-efe5f5b48977

In [5]:
!pip install confluent-kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 35.5 MB/s eta 0:00:00


In [6]:
from confluent_kafka import Producer
import json
import time
import pandas as pd



In [7]:
df = pd.read_csv('/content/data_test_kafka/first_100_rows.csv')
df.count()

,0
customer_id,99
name,99
city,99
state,99
country,99
registration_date,99
is_active,99


In [8]:
json_output = df.to_dict(orient='records')

json_file = 'customer.json'

with open(json_file,'w') as f:
  json.dump(json_output,f,indent=4)

print('Data converted to json and save in customer.json')

Data converted to json and save in customer.json


In [ ]:
# code to extract first 100 records from the big csv: (Ignore)

In [2]:
# @title
import csv

def extract_and_save_first_n_rows(input_csv_filepath, output_csv_filepath, n=100):
    """Extracts the first n rows from an input CSV and saves them to a new CSV.

    Args:
        input_csv_filepath (str): The path to the input CSV file.
        output_csv_filepath (str): The path to the output CSV file.
        n (int, optional): The number of rows to extract. Defaults to 100.
    """
    try:
        with open(input_csv_filepath, 'r', newline='') as infile, \
             open(output_csv_filepath, 'w', newline='') as outfile:
            reader = csv.reader(infile)
            writer = csv.writer(outfile)
            for i, row in enumerate(reader):
                if i < n:
                    writer.writerow(row)
                else:
                    break
        print(f"Successfully extracted the first {n} rows and saved to {output_csv_filepath}")
    except FileNotFoundError:
        print(f"Error: Input file not found at {input_csv_filepath}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
input_file = '/content/data_test_kafka/customers.csv'  # Replace with the actual path to your input CSV file
output_file = '/content/data_test_kafka/first_100_rows.csv'  # Replace with the desired path for the output CSV file
extract_and_save_first_n_rows(input_file, output_file)

Successfully extracted the first 100 rows and saved to /content/data_test_kafka/first_100_rows.csv


In [9]:
from confluent_kafka import Producer
import json
import time

config = {
"bootstrap.servers":"pkc-619z3.us-east1.gcp.confluent.cloud:9092",
"security.protocol":"SASL_SSL",
"sasl.mechanisms":"PLAIN",
"sasl.username":"7K4SVPMLQXL5AL3A",
"sasl.password":"5XTw3jiNzKiLQLmv0nAmqbQvwgQS/BKD1yF9NchOWAZrj6NrPtMsz5LoZ7DPZ9Vt",
"session.timeout.ms":45000,
"client.id":"ccloud-python-client-b18b45dd-cd5b-4861-83c8-efe5f5b48977"
}

producer = Producer(config)

In [10]:
topic = 'topic_0'

with open ('customer.json') as f:
  customers = json.load(f)

In [11]:
value = customers[0]
key = value['customer_id']

print (key,value)

0 {'customer_id': 0, 'name': 'Customer_0', 'city': 'Pune', 'state': 'Maharashtra', 'country': 'India', 'registration_date': '2023-06-29', 'is_active': False}


In [12]:
type(str(value).encode('utf-8'))

bytes

In [19]:
producer.produce(topic,key=str(key).encode('utf-8'),value=str(value).encode('utf-8'))

In [1]:
# sending multiple data to kafka

In [15]:
def delivery_status(errs,msg):
  if errs:
    print(f"Message delivery failed with error: {errs}")
  else:
    print(f"Message delivered successfully to topic: {msg.topic()} [{msg.partition()}] at offset: {msg.offset()}")


for record in customers:
  try:
    message_value = json.dumps(record)
    message_key = str(record['customer_id']).encode('utf-8')
    producer.produce(topic=topic,key=message_key,value=message_value,callback = delivery_status)
    producer.poll(1)
  except Exception as e:
    print (f"Error happened with : {e}")

producer.flush()
print("Message sent to kafka")


Message delivered successfully to topic: topic_0 [2] at offset: 2
Message delivered successfully to topic: topic_0 [2] at offset: 3
Message delivered successfully to topic: topic_0 [1] at offset: 0
Message delivered successfully to topic: topic_0 [1] at offset: 1
Message delivered successfully to topic: topic_0 [1] at offset: 2
Message delivered successfully to topic: topic_0 [1] at offset: 3
Message delivered successfully to topic: topic_0 [1] at offset: 4
Message delivered successfully to topic: topic_0 [0] at offset: 0
Message delivered successfully to topic: topic_0 [2] at offset: 4
Message delivered successfully to topic: topic_0 [0] at offset: 1
Message delivered successfully to topic: topic_0 [0] at offset: 2
Message delivered successfully to topic: topic_0 [0] at offset: 3
Message delivered successfully to topic: topic_0 [0] at offset: 4
Message delivered successfully to topic: topic_0 [2] at offset: 5
Message delivered successfully to topic: topic_0 [0] at offset: 5
Message de